# Problem 2
we mostly use Problem 1 decision variable, constrains, obj function and add more
## add parameters
* ContainerCapacity
* ContainerCost
* CBM_i be the cubic meter of per product CBM_i
## add decision varaible
* let ContainCnt_j be the container we buy at j month
## add ultility expression
* let $ExprZ_j$ be the volume sum of all product that are oreder in month $j$ (in cubic meter) by ocean<br>
$ExprZ_j = \sum_{i=0}^N x_{ijk} \times  CBM_i \quad \forall j$
## add to objective function
$(\sum_{j=0}^M ContainerCnt_j)\times ContainerCost$
## add constrains
$\dfrac{ExprZ_j}{ContainerCapacity} \le ContrainerCnt_j \quad \forall j$

In [7]:
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

In [8]:
## setting parameter
N, M, K = 10, 6, 3
D_ij = pd.read_excel('data.xlsx', sheet_name='Demand', index_col='Product').to_numpy()
I_i = pd.read_excel('data.xlsx', sheet_name='Initial inventory', index_col='Product').to_numpy().squeeze()
BuyCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Purchasing cost'].to_numpy().squeeze()
HoldCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Holding'].to_numpy().squeeze()
Transit_ij = pd.read_excel('data.xlsx', sheet_name='In-transit', index_col='Product').to_numpy().squeeze()
FixedShipCost_k = (100, 80, 50)
VarShipCost_ik = pd.read_excel('data.xlsx', sheet_name='Shipping cost', index_col='Product').to_numpy().squeeze()
x_max = sum([sum(i) for i in D_ij])
ContainerCapacity = 30
ContainerCost = 2750
CBM_i = pd.read_excel('data.xlsx', sheet_name='Size', index_col='Product').to_numpy().squeeze()

In [9]:
m = gb.Model("Problem2Model")

In [10]:
#add decision variable
x = m.addVars(N, M, K, vtype=GRB.INTEGER, name='x_ijk')
z = m.addVars(M, K, vtype=GRB.BINARY, name='z_jk')
ContainerCnt = m.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt_j')

In [11]:
#add ExprZ_j, ExprY_ij
ExprY_ij = [
    [
        gb.LinExpr(I_i[i] - D_ij[i][0]),
        gb.LinExpr(I_i[i] - sum(D_ij[i][z] for z in range(0, 2)) + x[i,0,0] + Transit_ij[i][1]),
        gb.LinExpr(I_i[i] - sum(D_ij[i][z] for z in range(0, 3)) + sum(x[i,z,0] for z in range(0, 2)) + sum(x[i,z,1] for z in range(0, 1)) +  sum(Transit_ij[i][z] for z in range(1, 3)) )
     ] +
    [
        gb.LinExpr(
            I_i[i] - sum(D_ij[i][z] for z in  range(0, j+1)) +
            sum(x[i,z,0] for z in range(0, j)) +
            sum(x[i,z,1] for z in range(0, j-1)) +
            sum(x[i,z,2] for z in range(0, j-2)) +
            Transit_ij[i][1] + Transit_ij[i][2]
        )
    for j in range(3, M)
    ]
for i in range(N)]

ExprZ_j = [
    quicksum(x[i,j,2]*CBM_i[i] for i in range(N))
for j in range(0, M)]

In [12]:
#add to objctive function
purchaseCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for k in range(0, K))
        for j in range(0, M))
    for i in range(0, N))
)

FixedCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            z[j,k] * FixedShipCost_k[k]
        for k in range(0, K))
    for j in range(0, M))
)

VariableCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * VarShipCost_ik[i][k]
            for k in range(K))
        for i in range(N))
    for j in range(M))
)

InventoryCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            ExprY_ij[i][j]
        for j in range(0, M)) * HoldCost_i[i]
    for i in range(0, N))
)

allContainerCostExpr =  gb.LinExpr(
    quicksum(ContainerCnt[j] * ContainerCost for j in range(M))
)

m.setObjective(
    purchaseCostExpr + FixedCostExpr + VariableCostExpr + InventoryCostExpr + allContainerCostExpr
)

In [13]:
#Constrains
_ = m.addConstrs(
    ExprY_ij[i][j] >= 0
for i in range(0, N)
for j in range(0, M)
)

_ = m.addConstrs(
    quicksum(x[i,j,k] for i in range(0, N)) / x_max <= z[j,k]
for j in range(0, M)
for k in range(0, K)
)
_ = m.addConstrs(
     ExprZ_j[j] / ContainerCapacity <= ContainerCnt[j]
 for j in range(M))

In [14]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 84 rows, 204 columns and 574 nonzeros
Model fingerprint: 0xf306d839
Variable types: 0 continuous, 204 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [5e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 7e+02]
Found heuristic solution: objective 1.700974e+07
Presolve removed 52 rows and 70 columns
Presolve time: 0.00s
Presolved: 32 rows, 134 columns, 334 nonzeros
Variable types: 0 continuous, 134 integer (11 binary)

Root relaxation: objective 1.650626e+07, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6506e+07    0    7 1.7010e+07 1.6506e+07  2.96%     -    0s
H    0     0                    1.650965e+07 1.6506e+07  0

In [15]:
for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

x_ijk[0,0,0] -0
x_ijk[0,0,1] -0
x_ijk[0,0,2] -0
x_ijk[0,1,0] -0
x_ijk[0,1,1] -0
x_ijk[0,1,2] -0
x_ijk[0,2,0] -0
x_ijk[0,2,1] -0
x_ijk[0,2,2] 38
x_ijk[0,3,0] -0
x_ijk[0,3,1] -0
x_ijk[0,3,2] -0
x_ijk[0,4,0] -0
x_ijk[0,4,1] -0
x_ijk[0,4,2] -0
x_ijk[0,5,0] -0
x_ijk[0,5,1] -0
x_ijk[0,5,2] -0
x_ijk[1,0,0] -0
x_ijk[1,0,1] -0
x_ijk[1,0,2] -0
x_ijk[1,1,0] -0
x_ijk[1,1,1] -0
x_ijk[1,1,2] -0
x_ijk[1,2,0] -0
x_ijk[1,2,1] -0
x_ijk[1,2,2] 45
x_ijk[1,3,0] -0
x_ijk[1,3,1] -0
x_ijk[1,3,2] -0
x_ijk[1,4,0] -0
x_ijk[1,4,1] -0
x_ijk[1,4,2] -0
x_ijk[1,5,0] -0
x_ijk[1,5,1] -0
x_ijk[1,5,2] -0
x_ijk[2,0,0] -0
x_ijk[2,0,1] -0
x_ijk[2,0,2] -0
x_ijk[2,1,0] -0
x_ijk[2,1,1] -0
x_ijk[2,1,2] 82
x_ijk[2,2,0] -0
x_ijk[2,2,1] -0
x_ijk[2,2,2] 200
x_ijk[2,3,0] -0
x_ijk[2,3,1] -0
x_ijk[2,3,2] -0
x_ijk[2,4,0] -0
x_ijk[2,4,1] -0
x_ijk[2,4,2] -0
x_ijk[2,5,0] -0
x_ijk[2,5,1] -0
x_ijk[2,5,2] -0
x_ijk[3,0,0] -0
x_ijk[3,0,1] -0
x_ijk[3,0,2] -0
x_ijk[3,1,0] -0
x_ijk[3,1,1] -0
x_ijk[3,1,2] 97
x_ijk[3,2,0] -0
x_ijk[3,2,1] -0
x_ijk[3

In [16]:
for i in range(N):
    for j in range(M):
        print(f'{i},{j}: {ExprY_ij[i][j].getValue()}')

0,0: 662.0
0,1: 607.0
0,2: 435.0
0,3: 241.0
0,4: 147.0
0,5: 0.0
1,0: 410.0
1,1: 357.0
1,2: 289.0
1,3: 104.0
1,4: 91.0
1,5: 0.0
2,0: 346.0
2,1: 167.0
2,2: 166.0
2,3: 117.0
2,4: 0.0
2,5: 0.0
3,0: 208.0
3,1: 258.0
3,2: 180.0
3,3: 49.0
3,4: 0.0
3,5: 0.0
4,0: 365.0
4,1: 303.0
4,2: 220.0
4,3: 130.0
4,4: 0.0
4,5: 0.0
5,0: 433.0
5,1: 356.0
5,2: 272.0
5,3: 145.0
5,4: 29.0
5,5: 0.0
6,0: 348.0
6,1: 212.0
6,2: 238.0
6,3: 122.0
6,4: 3.0
6,5: 0.0
7,0: 181.0
7,1: 26.0
7,2: 16.0
7,3: 0.0
7,4: 0.0
7,5: 0.0
8,0: 565.0
8,1: 489.0
8,2: 335.0
8,3: 159.0
8,4: 78.0
8,5: 0.0
9,0: 154.0
9,1: 0.0
9,2: 0.0
9,3: 0.0
9,4: 0.0
9,5: 0.0


In [17]:
print(f'purchaseCost: {purchaseCostExpr.getValue()}')
print(f'FixedCost: {FixedCostExpr.getValue()}')
print(f'VariableCost: {VariableCostExpr.getValue()}')
print(f'InventoryCost: {InventoryCostExpr.getValue()}')
print(f'allContainerCost: {allContainerCostExpr.getValue()}')


purchaseCost: 15232000.0
FixedCost: 410.0
VariableCost: 6776.0
InventoryCost: 1256400.0
allContainerCost: 13750.0
